# 1-2 - RegexTree

*Sébastien Meyer, Marie Tcheng*

Dans cet atelier, vous allez écrire des fonctions pour segmenter un texte juridique en vous basant sur les formats habituels d'écriture de ces textes.

Voici un exemple d'un tel texte : https://www.georisques.gouv.fr/webappReport/ws/installations/document/cf8c26fa8e9248118f91dd444a11b763

Dans un texte juridique de type "arrêté préfectoral", on retrouve habituellement la structure suivante : 

- En-tête : contient éventuellement la devise nationale, le titre du l'arrêté, l'autorité qui prend l'arrêté...

Exemple : 

```
ARRETE ARS GRAND EST n 2025-0182 du 13 janvier 2025
Portant fixation du bilan quantitatif de l'offre de soins pour la période de dépôt des demandes d'autorisation des activités de soins ouverte du 1er février 2025 au 1er avril 2025 pour la région Grand Est
La Directrice Générale de l'Agence Régionale de Santé Grand Est
```

- Visas : ce sont les textes visés par cet arrêté qui s'appliquent en premier lieu

Exemple :

```
- VU le Code de la santé publique, et notamment ses articles L 6122-9 et R 6122-30.
- VU le décret du 21 mai 2024 portant nomination de Mme Christelle RATIGNIER CARBONNEIL en qualité de la directrice Générale de l'Agence Régionale de Santé Grand Est.
```

- Motifs : ce sont les raisons pour lesquelles l'arrêté a été pris

Exemple : 

```
CONSIDÉRANT les risques sur certains sentiers de randonnée pédestre situés sur le domaine forestier géré par l'ONF de La Réunion à la suite du passage du cyclone Garance ;
CONSIDÉRANT qu'il y a lieu d'assurer la sécurité publique ;
```

- Titre : un "Arrête" en lettres capitales symbolise le début des prescriptions

Exemple : 

```
A R R E T E 
```

- Corps de texte : ce sont des articles qui se suivent, chaque article peut contenir des sous-articles et/ou des alinéas. Un alinéa définit des prescriptions, il s'agit en règle générale d'un paragraphe bien qu'en réalité ce soit plus subtil 

Exemple : 

```
Article 1er

La circulation de tous les véhicules à moteur est interdite sur :

- l'autoroute A1, dans les deux sens de circulation entre la jonction des échangeurs A1/A21 à Dourges et le tronc commun A1/22 à Lille ;

Article 2

Sur l'A1, l'ensemble des bretelles d'insertion sur l'autoroute A1 sont fermées entre la jonction des échangeurs A1/A21 à Dourges et le tronc commun A1/22 à Lille dans le sens Paris vers Lille. La bretelle d'insertion est fermée à l'échangeur 20 (Lesquin) dans le sens Lille vers Paris.
```

- Signature : il y a toujours une signature de l'autorité compétente qui prend l'arrêté 

Exemple : 

```
Pour le préfet de zone et par délégation, Le préfet délégué pour la défense et la sécurité
Vincent LASOGUEY
```

- Annexes : il peut s'agir de tableaux, de plans, etc.

Exemple : 

```
ANNEXE 1 : modèle du bulletin de vote 
Modèle de bulletin de vote
```

**NB** : Dans les visas et le corps de texte, on peut retrouver des citations vers d'autres textes juridiques. Ainsi, il est important de pouvoir faire un lien entre un texte juridique et les textes sur lesquels il s'appuie.

## Imports initiaux

Vous ne devez pas modifier cette partie du code. Il s'agit du nécessaire pour que vous puissiez remplir l'exercice.

In [ ]:
import pytest
import arretify.step_segmentation.titles_detection 
import arretify.step_references_detection.codes_detection
import arretify.step_references_detection.decrets_detection

from arretify.regex_utils import Settings, regex_tree
from arretify.parsing_utils.dates import DATE_NODE 
from arretify.step_references_detection.arretes_detection import IDENTIFIER_NODE

In [ ]:
def run_test_selection(test_file_name, test_class_name, test_method_name):
    pytest.main(["-vv", "--pyargs", f"{test_file_name}::{test_class_name}::{test_method_name}"])

## Introduction des concepts utilisés

Pour réaliser le parsing de structures compliquées, l'utilisation d'un moteur simple d'expressions régulières (regex) ne suffit pas toujours. Pour ce faire, la librairie Arrêtify développée par la DGPR comporte un moteur appelé *regex_tree*. 

On appelle "contenu matché" le contenu d'une phrase qui correpond à la recherche regex.

Ce moteur regroupe plusieurs fonctions qui permettent de réaliser des actions de regex tout en ajoutant des options : 

- *Group* : tout le contenu matché est associé à ce groupe auquel on donne un nom avec l'argument *group_name*. Cette classe est surtout utilisée pour des traitements internes, elle ne sera pas utile pour les exercices, par contre chaque regex finale doit être contenue dans un *Group*. 

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    r"article \d+",
    group_name="titre"
)
``` 

- *Branching* : c'est l'équivalent du caractère "|" dans une regex et elle permet de créer un branchement, dans lequel chaque regex est testée à la suite et on s'arrête au premier cas de match 

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Branching(
        [
            r"article 1",
            r"article 2",
        ],
    )
    group_name="titre"
)
```

- *Sequence* : prend en argument une liste de regex pour découper un texte dans un ordre prédéfini

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Sequence(
        [
            r"article ",
            r"\d+",
        ],
    )
    group_name="titre"
)
```

- *Literal* : contient une regex simple et permet d'y adjoindre des paramètres spécifiques (voir ci-dessous)

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Literal(
        r"article \d+",
        settings=Settings(),
    )
    group_name="titre"
)
```

- *Repeat* : réalise un dénombrement des éléments dans une regex "{n,p}"

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Repeat(
        r" ",
        quantifier=(0, 1),
    )
    group_name="titre"
)
```

**Note** : tous les éléments de *regex_tree* peuvent prendre en argument un élément Settings qui peut contenir des paramètres suivants : 
- *ignore_case* : ignorer ou non de la casse par exemple "ArTiCLe" (par défaut *true*) 
- *ignore_accents* : ignorer ou non des accents (par défaut *true*)
- *normalize_quotes* : agir de la même façon quel que soit le type de guillement (par défaut *true*)
- *normalize_dashes* : agir de la même façon quel que soit le type de tiret (par défaut *true*)

## Exercice 1 - détection des codes

Dans la législation française, il existe une liste finie de codes qui regroupent un ensemble de concepts législatifs et réglementaires. 

Nous sommes intéressés par reconnaître les codes suivants : 
- code du travail
- code de l'environnement
- code minier
- code forestier 

Construisez un détecteur de code en vous basant sur *regex_tree*. Il est interdit d'utiliser le signe "|" dans vos regex. Votre regex doit extraire le nom du code dans une balise *title*. 

In [ ]:
code_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="code"
)

arretify.step_references_detection.codes_detection.CODE_NODE = code_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_references_detection.codes_detection_test",
    "TestParseCodesReferences",
    "test_simple"
)

## Exercice 2 - détection des décrets

Il existe plusieurs types de décrets qui sont donc numérotés avec des chiffres éventuellemen séparés par des tirets.

*Exemple* : 

```
décret n°2005-635 du 30 mai 2005
```

Construisez une regex pour détecter les décrets. Le numéro doit être extrait dans une balise *identifier*. 

**NB** : Dans Arrêtify, il existe déjà un groupe *DATE_NODE* conçu pour reconnaître les dates que vous pouvez utiliser directement.

In [ ]:
decret_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="__decret",
)

arretify.step_detection.decrets_detection.DECRET_NODE = decret_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_references_detection.decrets_detection_test",
    "TestParseDecretsReferences",
    "test_simple"
)

## Exercice 3 - détection des arrêtés

Il existe de nombreux types d'arrêtés en France : ministériel, préfectoral, sans dénomination particulière.

Exemples : 

```
arrêté du 2 février 1998
arrêté ministériel du 2 février 1998
arrêtés préfectoraux n° 5213 du 28 octobre 1988 et n° 1636 du 24/03/95
```

Construisez une regex pour détecter les arrêtés au singulier.

Construisez une regex pour détecter les arrêtés au pluriel avec leur numérotation extraite dans une balise *identifier*.

**NB** : Pour la numérotation des arrêtés (ex : "n°3404"), vous pouvez vous appuyer sur le groupe *IDENTIFIER_NODE* existant dans Arrêtify.

Pour les arrêtés au singulier

In [ ]:
arrete_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="__arrete",
)

arretify.step_detection.arretes_detection.ARRETE_NODE = arrete_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_references_detection.arretes_detection_test",
    "TestParseArreteReferences",
    "test_arrete_unknown"
)

In [ ]:
run_test_selection(
    "arretify.step_references_detection.arretes_detection_test",
    "TestParseArreteReferences",
    "test_arrete_date1"
)

Et pour les arrêtés au pluriel

In [ ]:
arrete_multiple_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="__arrete_multiple",
)

arretify.step_detection.arretes_detection.ARRETE_MULTIPLE_NODE = arrete_multiple_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_references_detection.arretes_detection_test",
    "TestParseArretePluralReferences",
    "test_references_multiple"
)

## Exercice 4 - détection des articles

Il existe plusieurs formats de titres selon les arrêtés. Les parties de code suivantes permettent de petit à petit construire un module qui détecte les différents titres tout en évitant de mal détecter d'autres éléments qui n'en sont pas. 

Nous allons d'abord construire des briques de détection avant d'assembler les tests.

La librairie Arrêtify définit un titre de la manière suivante : 
- *section_type* : titre, chapitre, article, annexe, unknown, alinea
- *number* : le numéro dans le titre
- *levels* : le numéro du titre découpé dans une liste
- *test* : le texte suivant le numéro du titre

*Exemple* : Cet objet représente la phrase suivante : "Article 1.1 - Objet".

```python
title_example = TitleInfo(
    section_type=SectionType.ARTICLE,
    number="1.1",
    levels=[1, 1],
    text="Objet",
)
```

**NB** : Vous pouvez vous aider de groupes déjà construits dans Arrêtify directement dans vos regex : 
- **SECTION_TYPES_PATTERN_S** : une regex avec les différents *section_type*
- **NUMBERING_THEN_OBL_NUMBERS_PATTERN_S** : une suite de nombres avec au moins 2 nombres (ex : "1.1")
- **NUMBERING_THEN_OPT_NUMBERS_PATTERN_S** : une suite de nombres avec au moins 1 nombre (ex : "1.")

### Cas 1

On souhaite détecter les cas suivants :

```
ARTICLE 1
```

Le résultat doit être :

```python
TitleInfo(
    section_type=SectionType.ARTICLE,
    number="1",
    levels=[1],
)
```

Construisez une regex pour ce cas. Vous devez extraire les éléments *section_type*, *punc_before*, *number*, *punc_after*, *text*. Les éléments *punc_before* et *punc_after* entourent la numérotation. 

In [ ]:
title_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="title"
)

arretify.step_segmentation.titles_detection.TITLE_NODE = title_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_article_with_arabic_number_without_dot"
)

### Cas 2

On souhaite également détecter les cas suivants : 

```
TITRE I - Premier titre
Chapitre A - Premier chapitre
ARTICLE 1.1.1 Premier article
```

Les résultats doivent être : 

```python
TitleInfo(
    section_type=SectionType.TITRE,
    number="1",
    levels=[1],
    text="Premier titre",
)

TitleInfo(
    section_type=SectionType.CHAPITRE,
    number="A",
    levels=[1],
    text="Premier chapitre",
)

TitleInfo(
    section_type=SectionType.ARTICLE,
    number="1.1.1",
    levels=[1, 1, 1],
    text="Premier article",
)
```

Construisez une regex pour ce cas. Vous devez extraire les éléments *section_type*, *punc_before*, *number*, *punc_after*, *text*. Les éléments *punc_before* et *punc_after* entourent la numérotation. 

In [ ]:
title_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="title"
)

arretify.step_segmentation.titles_detection.TITLE_NODE = title_node

Utilisez les cellules suivantes pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_title_with_roman_number"
)

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_chapter_with_letter_without_dot"
)

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_hierarchical_article_with_arabic_number_without_dot"
)

### Cas complet 

On souhaite maintenant couvrir tous les cas suivants en plus des précédents : 

```
# Numérotation avant le titre 
1.1.1 Article
# Numérotation séparée par un tiret
Article 2-1 Article
# Numérotation romaine
II.1.1 - Sous-titre
# Annexe
ANNEXÉ 3 - Attestation du Prestataire
```

Les résultats doivent être : 

```python
TitleInfo(
    section_type=SectionType.ARTICLE,
    number="1.1.1",
    levels=[1, 1, 1],
)

TitleInfo(
    section_type=SectionType.ARTICLE,
    number="2-1",
    levels=[2, 1],
    text="Article",
)

TitleInfo(
    section_type=SectionType.ARTICLE,
    number="II.1.1",
    levels=[2, 1, 1],
    text="Sous-titre",
)

TitleInfo(
    section_type=SectionType.ANNEXE,
    number="3",
    levels=[3],
    text="Attestation du Prestataire",
)
```

Construisez une regex pour ce cas. Vous devez extraire les éléments *section_type*, *punc_before*, *number*, *punc_after*, *text*. Les éléments *punc_before* et *punc_after* entourent la numérotation. 

In [ ]:
title_node = regex_tree.Group(
    """VOTRE CODE ICI""",
    group_name="title"
)

arretify.step_segmentation.titles_detection.TITLE_NODE = title_node

Utilisez les cellules suivantes pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_hierarchical_title_no_name"
)

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_hyphen_in_numbering"
)

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_roman_no_section_name"
)

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_title_with_wrongly_accented_char"
)


-----